In [ ]:
!pip install transformers datasets torchinfo seqeval

# Download and EDA dataset

In [2]:
from datasets import load_dataset

# Download conll2003 dataset
data = load_dataset('conll2003')

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
data['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [5]:
data['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None),
 'chunk_tags': Sequence(feature=ClassLabel(names=['O', 'B-ADJP', 'I-ADJP', 'B-ADVP', 'I-ADVP', 'B-CONJP', 'I-CONJP', 'B-INTJ', 'I-INTJ', 'B-LST', 'I-LST', 'B-NP', 'I-NP', 'B-PP', 'I-PP', 'B-PRT', 'I-PRT', 'B-SBAR', 'I-SBAR', 'B-UCP', 'I-UCP', 'B-VP', 'I-VP'], id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)}

In [6]:
print(data['train'].features['ner_tags'])

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)


In [7]:
label_names = data['train'].features['ner_tags'].feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

# Tokenizer

In [8]:
from transformers import AutoTokenizer

# Load tokenizer form checkpoint
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [9]:
idx = 0
example = data['train'][idx]['tokens']
example

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [10]:
t = tokenizer(example, is_split_into_words=True)
t

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
t.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [12]:
t.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [13]:
label_names[0]

'O'

In [14]:
# Align and label special tokens and subword tokens
begin2inside ={
    1:2,
    3:4,
    5:6,
    7:8
}
def align_targets(labels, word_ids):
  aligned_labels = []
  last_word_id = None
  for word_id in word_ids:
    if word_id is None:
      label = -100 # For special tokens
    elif word_id != last_word_id:
      label = labels[word_id] # For a new word
    else:
      # For the same word as before
      label = labels[word_id]
      # Change B-<tag> to I-<tag> if necessary
      if label in begin2inside:
        label = begin2inside[labels[word_id]]
    # Add the label
    aligned_labels.append(label)
    # Update last word 
    last_word_id = word_id

  return aligned_labels


In [15]:
# Test align_targets function
labels = data['train'][idx]['ner_tags']
word_ids = t.word_ids()
aligned_targets = align_targets(labels, word_ids)
aligned_targets

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]

In [16]:
aligned_labels = [label_names[t] if t>= 0 else None for t in aligned_targets]
aligned_labels

[None, 'B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', None]

In [17]:
words = [
    '[CLS]', 'Ger', '##man', 'call', 'to', 'boycott', 'Micro', '##soft', '[SEP]'
]
word_ids = [None, 0, 0, 1, 2, 3, 4, 4, None]
labels = [7, 0, 0, 0, 3]

aligned_targets = align_targets(labels, word_ids)
aligned_labels = [label_names[t] if t >= 0 else None for t in aligned_targets]
for x, y in zip(words, aligned_labels):
  print(f"{x}\t{y}")

[CLS]	None
Ger	B-MISC
##man	I-MISC
call	O
to	O
boycott	O
Micro	B-ORG
##soft	I-ORG
[SEP]	None


In [18]:
# tokenize both the inputs and targets
def tokenize_fn(batch):
  # This returns input_ids, attention_mask
  tokenized_inputs = tokenizer(
      batch['tokens'], truncation=True, is_split_into_words=True
  )
  labels_batch = batch['ner_tags'] # Origin targets
  aligned_labels_batch = []
  for i, labels in enumerate(labels_batch):
    word_ids = tokenized_inputs.word_ids(i)
    aligned_labels_batch.append(align_targets(labels, word_ids))
  # Targets must be stored in key called 'labels'
  tokenized_inputs['labels'] = aligned_labels_batch
  return tokenized_inputs

In [19]:
# mapping and remove unnecessary columns 
tokenized_ds = data.map(
    tokenize_fn,
    batched=True,
    remove_columns=data['train'].column_names
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [20]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [21]:
from transformers import DataCollatorForTokenClassification
# Doing the padding and conversion into torch tensor only on current batch
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [22]:
from datasets import load_metric
# Load metric for NLP tasks with sequence targets
metric = load_metric('seqeval')

<ipython-input-22-c876b65aca5e>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('seqeval')


In [23]:
# Test metric. predictions and references should be the list of the IAB format list
metric.compute(
    predictions=[['O','O','I-ORG', 'B-MISC']],
    references=[['O','B-ORG','I-ORG', 'B-MISC']]
)

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ORG': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'overall_precision': 0.5,
 'overall_recall': 0.5,
 'overall_f1': 0.5,
 'overall_accuracy': 0.75}

In [24]:
import numpy as np

def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  preds = np.argmax(logits, axis=-1)

  # Remove -100 from labels and predictions 
  # Convert the label_ids to label name
  str_labels = [
      [label_names[t] for t in label if t != -100] for label in labels
  ]
  str_preds = [
      [label_names[p] for p, t in zip(pred, target) if t != -100] for pred, target in zip(preds, labels) 
  ]

  
  the_metrics = metric.compute(predictions=str_preds, references=str_labels)

  return {
      'precision': the_metrics['overall_precision'],
      'racall': the_metrics['overall_recall'],
      'f1': the_metrics['overall_f1'],
      'accuracy': the_metrics['overall_accuracy']
  }

In [25]:
id2label = {k:v for k, v in enumerate(label_names)}
label2id = {v:k for k, v in id2label.items()}

In [26]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 

In [44]:
from transformers import TrainingArguments

checkpoint_path = 'NER/checkpoint/'
training_args = TrainingArguments(
    checkpoint,
    per_device_train_batch_size=16,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [45]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [46]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14041
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4390
  Number of trainable parameters = 65197833


Epoch,Training Loss,Validation Loss,Precision,Racall,F1,Accuracy
1,0.004300,0.134494,0.898751,0.932178,0.915159,0.980058
2,0.003200,0.117981,0.910340,0.936385,0.923179,0.982295
3,0.002200,0.118037,0.916858,0.940929,0.928738,0.983119
4,0.001700,0.120049,0.916544,0.940761,0.928494,0.983414
5,0.000600,0.117243,0.921061,0.940592,0.930724,0.983664


***** Running Evaluation *****
  Num examples = 3250
  Batch size = 8
Saving model checkpoint to distilbert-base-cased/checkpoint-878
Configuration saved in distilbert-base-cased/checkpoint-878/config.json
Model weights saved in distilbert-base-cased/checkpoint-878/pytorch_model.bin
tokenizer config file saved in distilbert-base-cased/checkpoint-878/tokenizer_config.json
Special tokens file saved in distilbert-base-cased/checkpoint-878/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 8
Saving model checkpoint to distilbert-base-cased/checkpoint-1756
Configuration saved in distilbert-base-cased/checkpoint-1756/config.json
Model weights saved in distilbert-base-cased/checkpoint-1756/pytorch_model.bin
tokenizer config file saved in distilbert-base-cased/checkpoint-1756/tokenizer_config.json
Special tokens file saved in distilbert-base-cased/checkpoint-1756/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3250
  Batch size =

TrainOutput(global_step=4390, training_loss=0.002513797136537035, metrics={'train_runtime': 461.3101, 'train_samples_per_second': 152.186, 'train_steps_per_second': 9.516, 'total_flos': 877156941780630.0, 'train_loss': 0.002513797136537035, 'epoch': 5.0})

Model has accuracy and f1 high after 5 epochs

In [47]:
save_path = 'NER'
trainer.save_model(save_path)

Saving model checkpoint to NER
Configuration saved in NER/config.json
Model weights saved in NER/pytorch_model.bin
tokenizer config file saved in NER/tokenizer_config.json
Special tokens file saved in NER/special_tokens_map.json


In [48]:
from transformers import pipeline

ner = pipeline(
    'token-classification',
    model=save_path,
    aggregation_strategy='simple'
)

loading configuration file NER/config.json
Model config DistilBertConfig {
  "_name_or_path": "NER",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-MISC",
    "8": "I-MISC"
  },
  "initializer_range": 0.02,
  "label2id": {
    "B-LOC": 5,
    "B-MISC": 7,
    "B-ORG": 3,
    "B-PER": 1,
    "I-LOC": 6,
    "I-MISC": 8,
    "I-ORG": 4,
    "I-PER": 2,
    "O": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "vocab_size": 28996
}

loading configurat

In [56]:
ner(data['test'][1]['tokens'])

[[{'entity_group': 'PER',
   'score': 0.9993741,
   'word': 'Nadim',
   'start': 0,
   'end': 5}],
 [{'entity_group': 'PER',
   'score': 0.99995357,
   'word': 'Ladki',
   'start': 0,
   'end': 5}]]

In [50]:
ner("Vietnam is a country in Southeast Asia. The capital is Hanoi and the great leader is Ho Chi Minh")

[{'entity_group': 'LOC',
  'score': 0.9999851,
  'word': 'Vietnam',
  'start': 0,
  'end': 7},
 {'entity_group': 'LOC',
  'score': 0.99995196,
  'word': 'Southeast Asia',
  'start': 24,
  'end': 38},
 {'entity_group': 'LOC',
  'score': 0.9999299,
  'word': 'Hanoi',
  'start': 55,
  'end': 60},
 {'entity_group': 'PER',
  'score': 0.99996537,
  'word': 'Ho Chi Minh',
  'start': 85,
  'end': 96}]

In [57]:
ner("Viettel is a big corporation in Vietnam")

[{'entity_group': 'ORG',
  'score': 0.9999693,
  'word': 'Viettel',
  'start': 0,
  'end': 7},
 {'entity_group': 'LOC',
  'score': 0.9999492,
  'word': 'Vietnam',
  'start': 32,
  'end': 39}]